In [274]:
from lnn import Model, Predicate, Variable, Variables, And, Or, Not, Predicates, Fact, Loss, Propositions, Implies, World, Variables
import gym
import numpy as np
import torch

In [2]:
env = gym.make("CartPole-v1")

In [85]:
model = Model()

p1, p2, p3 = Propositions("p1", "p2", "p3")
a1 = And(p1, p2)
o1 = Or(p1, p2)
a2 = And(a1, o1)
i1 = Implies(a2, p3, world= World.AXIOM)

In [5]:
model.print()


***************************************************************************
                                LNN Model

***************************************************************************


In [6]:
model.add_knowledge(a1, a2, o1, i1)
model.add_data({
    p1: Fact.TRUE,
    p2: Fact.TRUE
})


model.add_labels({
    p3: Fact.FALSE
})

model.train(losses = [Loss.SUPERVISED])
model.print(params = True)


***************************************************************************
                                LNN Model

AXIOM Implies: (((p1 ∧ p2) ∧ (p1 ∨ p2)) → p3)               TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Proposition: p3                                        TRUE (1.0, 1.0)
params  α: 1.0
OPEN And: ((p1 ∧ p2) ∧ (p1 ∨ p2))                           TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Or: (p1 ∨ p2)                                          TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN And: (p1 ∧ p2)                                         TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Proposition: p2                                        TRUE (1.0, 1.0)
params  α: 1.0
OPEN Proposition: p1                                        TRUE (1.0, 1.0)
params  α: 1.0
***************************************************************************


In [46]:
# construct the model from formulae
model = Model()
p1, p2 = Predicates("P1", "P2")
x, y = Variables("X", "Y")

a1 = And(p1(x), p2(x))
model.add_knowledge(a1)
# o1 = Or(p1(x), Not(p2(y)))
# formulae = [a1, o1]
# model.add_knowledge(*formulae, world=World.AXIOM)

model.add_data({
    p1: {
        '0': Fact.TRUE,
    },
    p2: {
        '1': Fact.TRUE
    },
    a1: {
        '1': Fact.TRUE,
        '0': Fact.TRUE
    }
})
model.add_labels({
    p1: {
        '1': Fact.TRUE
    },
    p2: {
        '0': Fact.FALSE
    }
})

model.train(losses = [Loss.SUPERVISED, Loss.CONTRADICTION])
model.print(params=True)


***************************************************************************
                                LNN Model

OPEN And: (P1(0) ∧ P2(0)) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
'1'                                                         TRUE (1.0, 1.0)
'0'                                                         TRUE (1.0, 1.0)

OPEN Predicate: P2 
params  α: 1.0
'1'                                                         TRUE (1.0, 1.0)
'0'                                                         TRUE (1.0, 1.0)

OPEN Predicate: P1 
params  α: 1.0
'0'                                                         TRUE (1.0, 1.0)
'1'                                                         TRUE (1.0, 1.0)

***************************************************************************


In [137]:
model.print()


***************************************************************************
                                LNN Model

OPEN And: (P1(0) ∧ P2(0)) 

OPEN Predicate: P2 

OPEN Predicate: P1 

***************************************************************************


In [47]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [61]:
maxvel = 0
maxangvel = 0
for i in range(100):
    env.reset()
    while True:
        next_state, reward, done, _, _ = env.step(0)
        if done:
            break
        if abs(next_state[1]) > maxvel:
            maxvel = abs(next_state[1])
        if abs(next_state[3]) > maxangvel:
            maxangvel = abs(next_state[3])
print(maxvel, maxangvel)

1.9889573 2.96616


In [64]:
velocities = np.linspace(-2, 2, 41)
angvelocities = np.linspace(-3, 3, 61)
pos = np.linspace(-1.2, 1.2, 25)
ang = np.linspace(-0.2094395, 0.2094395, 25)

In [65]:
model = Model()
velocity_variables = []
angvelocity_variables = []
pos_variables = []
ang_variables = []

In [66]:
for i in range(41):
  velocity_variables.append(Predicate("vel_"+str(i+1)))
for i in range(61):
  angvelocity_variables.append(Predicate("angvel_"+str(i+1)))
for i in range(25):
  pos_variables.append(Predicate("pos_"+str(i+1)))
for i in range(25):
  ang_variables.append(Predicate("ang_"+str(i+1)))

p_1 p_2 p_3 p_4 p_5 p_6 p_7 p_8 p_9 p_10


SyntaxError: starred assignment target must be in a list or tuple (2965991908.py, line 7)

In [453]:
def create_predicates(num_bins):
    predicate_list = []
    for i in range(num_bins):
        predicate_list.append(Predicate("p_" +str(i+1)))
    return predicate_list # ["p_1", "p_2", ...]

def create_pred_names(num_bins):
    return ["p_"+str(i+1) for i in range(num_bins)] # ["p_1", "p_2", ...]

def create_n_ary_and(num_nodes, predicate_list):
    and_list = []
    for i in range(num_nodes):
        and_list.append(And(*predicate_list))
    return and_list

def create_n_ary_or(and_list):
    return Or(*and_list)

def create_lnn(num_bins, num_nodes):
    model = Model()
#     preds = create_predicates(num_bins)
    names = create_pred_names(num_bins)
    preds = model.add_predicates(1, *names)
    and_nodes = create_n_ary_and(num_nodes, preds)
    or_node = create_n_ary_or(and_nodes)
    model.add_knowledge(*and_nodes)
    out = model.set_query(or_node)
    
    return model, preds, or_node

In [510]:
def create_predicates(num_bins):
    predicate_list = []
    for i in range(num_bins):
        predicate_list.append(Predicate("p_" +str(i+1)))
    return predicate_list # ["p_1", "p_2", ...]

def create_pred_names(num_bins):
    return ["p_"+str(i+1) for i in range(num_bins)] # ["p_1", "p_2", ...]

def create_n_ary_and(num_nodes, predicate_list):
    and_list = []
    for i in range(num_nodes):
        and_list.append(And(*predicate_list))
    return and_list

def create_n_ary_or(and_list):
    return Or(*and_list)

def create_lnn(num_bins, num_nodes):
    model = Model()
    preds = create_predicates(num_bins)
#     names = create_pred_names(num_bins)
#     preds = model.add_predicates(1, *names)
    and_nodes = create_n_ary_and(num_nodes, preds)
    or_node = create_n_ary_or(and_nodes)
    model.add_knowledge(*and_nodes)
    out = model.set_query(or_node)
    
    return model, preds, or_node

In [68]:
state = env.reset()

In [72]:
torch.tensor(state[0]).shape

torch.Size([4])

In [ ]:
def env_2_fol(obs):
    assert obs.shape = 

In [514]:
n_bins=3
n_ands=2
left = Model()
left, preds, out = create_lnn(n_bins, n_ands)
# print("out:", out)
# left.print()

d = dict(zip(preds, [{'0':Fact.FALSE} for _ in range(n_bins)]))
left.add_data({**d})
zeros = tuple('0' for _ in range(n_bins))
left.add_data({preds[0]:{'0':Fact.TRUE}})

d1 = dict(zip(preds, [{'1':Fact.FALSE} for _ in range(n_bins)]))
left.add_data({**d1})
ones = tuple('1' for _ in range(n_bins))
left.add_data({preds[0]:{'1':Fact.TRUE}})

left.add_labels({out: {zeros:Fact.TRUE, ones:Fact.TRUE}})
# left.print(params=True)
left.train(losses=[Loss.SUPERVISED], epochs=1)
left.print(params=True)
out.get_data()
# print(*d.keys())


***************************************************************************
                                LNN Model

OPEN Or: ((p_1(0) ∧ p_2(1) ∧ p_3(2)) ∨ (p_1(0) ∧ p_2(1) ∧ p_3(2))) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
('0', '0', '0')                                            FALSE (0.0, 0.0)
('0', '0', '1')                                            FALSE (0.0, 0.0)
('1', '1', '1')                                            FALSE (0.0, 0.0)
('1', '0', '1')                                            FALSE (0.0, 0.0)
('0', '1', '1')                                            FALSE (0.0, 0.0)
('0', '1', '0')                                            FALSE (0.0, 0.0)
('1', '0', '0')                                            FALSE (0.0, 0.0)
('1', '1', '0')                                            FALSE (0.0, 0.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [1.   0.95 0.95]
('0', '0', '0')                                            FALSE (0.0, 0.0)
('0', 

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<CopySlices>)

In [491]:
left.flush()
left.add_data({**d})
left.add_data({preds[0]:{'0':Fact.TRUE}})
left.add_labels({out: {zeros:Fact.TRUE}})
# left.print(params=True)
left.train(losses=[Loss.SUPERVISED], epochs=1)
left.print(params=True)
out.get_data(zeros)


***************************************************************************
                                LNN Model

OPEN Or: ((p_1(0) ∧ p_2(1) ∧ p_3(2)) ∨ (p_1(0) ∧ p_2(1) ∧ p_3(2))) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [1.   0.95 0.95]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [1.   0.95 0.95]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN Predicate: p_3 
params  α: 1.0
'0'                                                        FALSE (0.0, 0.0)

OPEN Predicate: p_2 
params  α: 1.0
'0'                                                        FALSE (0.0, 0.0)

OPEN Predicate: p_1 
params  α: 1.0
'0'                                                         TRUE (1.0, 1.0)

*************************************

tensor([[0., 0.]], grad_fn=<IndexBackward0>)

In [518]:
left.flush()
left.print(params=True)


***************************************************************************
                                LNN Model

OPEN Or: ((p_1(0) ∧ p_2(1) ∧ p_3(2)) ∨ (p_1(0) ∧ p_2(1) ∧ p_3(2))) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
('0', '0', '0')                                          UNKNOWN (0.0, 1.0)
('0', '0', '1')                                          UNKNOWN (0.0, 1.0)
('1', '1', '1')                                          UNKNOWN (0.0, 1.0)
('1', '0', '1')                                          UNKNOWN (0.0, 1.0)
('0', '1', '1')                                          UNKNOWN (0.0, 1.0)
('0', '1', '0')                                          UNKNOWN (0.0, 1.0)
('1', '0', '0')                                          UNKNOWN (0.0, 1.0)
('1', '1', '0')                                          UNKNOWN (0.0, 1.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [1.   0.95 0.95]
('0', '0', '0')                                          UNKNOWN (0.0, 1.0)
('0', 

In [489]:
left.flush()
# left.print(params=True)
left.add_data({**d})
left.add_data({preds[1]:{'0':Fact.TRUE}})
left.infer()
# left.print(params=True)
left.add_labels({out: {zeros:Fact.TRUE}})
left.train(losses=[Loss.SUPERVISED], epochs=1)
left.print(params=True)


***************************************************************************
                                LNN Model

OPEN Or: ((p_1(0) ∧ p_2(1) ∧ p_3(2)) ∨ (p_1(0) ∧ p_2(1) ∧ p_3(2))) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [0.65 0.75 0.7 ]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [0.65 0.75 0.7 ]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN Predicate: p_3 
params  α: 1.0
'0'                                                        FALSE (0.0, 0.0)

OPEN Predicate: p_2 
params  α: 1.0
'0'                                                         TRUE (1.0, 1.0)

OPEN Predicate: p_1 
params  α: 1.0
'0'                                                        FALSE (0.0, 0.0)

*************************************

In [485]:
left.flush()
# left.print(params=True)
left.add_data({**d})
left.add_data({preds[2]:{'0':Fact.TRUE}})
left.infer()
# left.print(params=True)
left.add_labels({out: {zeros:Fact.TRUE}})
left.train(losses=[Loss.SUPERVISED], epochs=1)
left.print(params=True)


***************************************************************************
                                LNN Model

OPEN Or: ((p_1(0) ∧ p_2(1) ∧ p_3(2)) ∨ (p_1(0) ∧ p_2(1) ∧ p_3(2))) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [0.8 0.8 0.9]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN And: (p_1(0) ∧ p_2(1) ∧ p_3(2)) 
params  α: 1.0,  β: 1.0,  w: [0.8 0.8 0.9]
('0', '0', '0')                                            FALSE (0.0, 0.0)

OPEN Predicate: p_3 
params  α: 1.0
'0'                                                         TRUE (1.0, 1.0)

OPEN Predicate: p_2 
params  α: 1.0
'0'                                                        FALSE (0.0, 0.0)

OPEN Predicate: p_1 
params  α: 1.0
'0'                                                        FALSE (0.0, 0.0)

*******************************************

In [93]:
print(model.infer_query())

None


In [180]:
test = create_predicates(10)

In [186]:
And(*test)